In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import glob
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve, auc, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier, MLPRegressor

In [2]:
insiders = pd.read_csv('qq_beta-live-insiders.csv')

In [34]:
prices = pd.read_csv('../data/av_query_ZWS.csv', skiprows=5)
prices['Date'] = pd.to_datetime(prices['5. Time Zone'])
prices.drop(columns=['US/Eastern', '5. Time Zone'], inplace=True)
prices.head()

,Unnamed: 2,Date
0,"{'1. open': '29.9500', '2. high': '30.0700', '...",2024-01-26
1,"{'1. open': '29.0200', '2. high': '29.8200', '...",2024-01-25
2,"{'1. open': '29.4300', '2. high': '29.4300', '...",2024-01-24
3,"{'1. open': '29.5500', '2. high': '29.6400', '...",2024-01-23
4,"{'1. open': '29.1500', '2. high': '29.5700', '...",2024-01-22


In [35]:
import json
prices['Open'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['1. open'].replace(',', '')))
prices['High'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['2. high'].replace(',', '')))
prices['Low'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['3. low'].replace(',', '')))
prices['Close'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['4. close'].replace(',', '')))
prices['Volume'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['5. volume'].replace(',', '')))
prices.drop(columns=['Unnamed: 2'], inplace=True)
prices.head()

,Date,Open,High,Low,Close,Volume
0,2024-01-26,29.95,30.07,29.48,29.68,1026479.0
1,2024-01-25,29.02,29.82,28.96,29.81,1190017.0
2,2024-01-24,29.43,29.43,28.53,28.60,1082118.0
3,2024-01-23,29.55,29.64,28.94,29.17,903778.0
4,2024-01-22,29.15,29.57,29.03,29.42,860426.0


In [38]:
insiders.dropna(axis=0, inplace=True)
insiders['Date'] = pd.to_datetime(insiders['Date'])
insiders['Ticker'] = insiders['Ticker'].astype(str)
insiders['Name'] = insiders['Name'].astype(str).str.lower()
insiders['fileDate'] = pd.to_datetime(insiders['fileDate'])
insiders['total_value'] = insiders['Shares'] * insiders['PricePerShare']
insiders.drop(insiders[insiders['Shares'] == 0].index, inplace=True)
insiders.drop(insiders[insiders['PricePerShare'] == 0].index, inplace=True)
insiders['log_total_value'] = np.log(insiders['total_value'])
insiders['TraderFrequency'] = insiders.groupby('Name')['Name'].transform('count')
insiders.head()

,Unnamed: 0,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency
0,0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,11.359463,2
1,1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,10.966688,2
2,2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,11.233259,150
3,3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,1.489031,150
4,4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,1.894497,150


In [39]:
insiders.drop(columns=['Unnamed: 0'], inplace=True)

In [40]:
insiders['change_in_holdings'] = (insiders['Shares'] / insiders['SharesOwnedFollowing']) * 100 
insiders.head()

,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency,change_in_holdings
0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,11.359463,2,2.124672
1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,10.966688,2,1.382447
2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,11.233259,150,0.124627
3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,1.489031,150,0.000007
4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,1.894497,150,0.000011


In [42]:

grpuped = insiders.groupby(['Name', 'Ticker', 'Date'])#.agg({'Shares': 'sum', 'PricePerShare': 'mean', 'SharesOwnedFollowing': 'mean', 'total_value': 'sum', 'TraderFrequency': 'mean', 'change_in_holdings': 'mean'})
grpuped.head()

,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency,change_in_holdings
0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,8.577333e+04,11.359463,2,2.124672
1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,5.791245e+04,10.966688,2,1.382447
2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,7.560362e+04,11.233259,150,0.124627
3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.432800e+00,1.489031,150,0.000007
4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.649200e+00,1.894497,150,0.000011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SFBC,2023-10-27,stilwell joseph,A,P,4470.0,35.7758,198342.0,2023-10-30 16:30:00,1.599178e+05,11.982415,35,2.253683
19996,NVR,2023-10-26,malzahn daniel david,D,S,356.0,5453.1000,10135.0,2023-10-30 16:03:15,1.941304e+06,14.478870,9,3.512580
19997,BOTJ,2023-10-26,alford john r jr,A,P,400.0,9.8500,21190.0,2023-10-30 15:48:47,3.940000e+03,8.278936,1,1.887683
19998,BUKS,2023-10-27,daly joseph patrick,A,P,5000.0,0.6800,3605000.0,2023-10-30 15:45:50,3.400000e+03,8.131531,45,0.138696


In [44]:
insiders['individual_transactions_per_trade'] = insiders.groupby(['Name', 'Ticker'])['Name'].transform('count')
insiders['investors_per_trade'] = insiders.groupby(['TransactionCode', 'Ticker'])['Name'].transform('nunique')
insiders.head()

,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency,change_in_holdings,individual_transactions_per_trade,investors_per_trade
0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,11.359463,2,2.124672,2,7
1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,10.966688,2,1.382447,2,7
2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,11.233259,150,0.124627,150,1
3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,1.489031,150,0.000007,150,1
4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,1.894497,150,0.000011,150,1


In [47]:
insiders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19815 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Ticker                             19815 non-null  object        
 1   Date                               19815 non-null  datetime64[ns]
 2   Name                               19815 non-null  object        
 3   AcquiredDisposedCode               19815 non-null  object        
 4   TransactionCode                    19815 non-null  object        
 5   Shares                             19815 non-null  float64       
 6   PricePerShare                      19815 non-null  float64       
 7   SharesOwnedFollowing               19815 non-null  float64       
 8   fileDate                           19815 non-null  datetime64[ns]
 9   total_value                        19815 non-null  float64       
 10  log_total_value                    1981

#### No need for an Imputer

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='median')

In [60]:
insiders.columns

Index([                           'Ticker',
                                    'Date',
                                    'Name',
                    'AcquiredDisposedCode',
                         'TransactionCode',
                                  'Shares',
                           'PricePerShare',
                    'SharesOwnedFollowing',
                                'fileDate',
                             'total_value',
                         'log_total_value',
                         'TraderFrequency',
                      'change_in_holdings',
       'individual_transactions_per_trade',
                     'investors_per_trade',
                                       'P',
                                       'S',
                                       'P',
                                       'S',
                                       nan,
                                       'P',
                                       'S',
                                

In [62]:
insiders = insiders.iloc[:, :-1]
insiders.head()

,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency,change_in_holdings,individual_transactions_per_trade,investors_per_trade,P
0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,11.359463,2.0,2.124672,2.0,7.0,0.0
1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,10.966688,2.0,1.382447,2.0,7.0,0.0
2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,11.233259,150.0,0.124627,150.0,1.0,0.0
3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,1.489031,150.0,0.000007,150.0,1.0,0.0
4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,1.894497,150.0,0.000011,150.0,1.0,0.0


In [51]:
onehot = OneHotEncoder(sparse=False)
transaction_onehot = onehot.fit_transform(insiders[['TransactionCode']])
transaction_onehot = pd.DataFrame(transaction_onehot, columns=onehot.categories_[0])
insiders = pd.concat([insiders, transaction_onehot], axis=1)
insiders.head()

/opt/anaconda3/envs/insider_algo/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Ticker,Date,Name,AcquiredDisposedCode,TransactionCode,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,...,individual_transactions_per_trade,investors_per_trade,P,S,P,S,NaN,P,S,NaN
0,NTRA,2023-12-29,brophy michael burkes,D,S,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,...,2.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,NTRA,2023-12-28,brophy michael burkes,D,S,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,...,2.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,EAF,2023-12-29,bcp gp ltd,D,S,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,...,150.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,EAF,2023-12-29,bcp gp ltd,D,S,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,...,150.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,EAF,2023-12-29,bcp gp ltd,D,S,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,...,150.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [64]:
insiders['Purchase?'] = insiders['P']
insiders.drop(columns=['P'], inplace=True)
insiders.drop(columns=['TransactionCode', 'AcquiredDisposedCode'], inplace=True)
insiders.head()

,Ticker,Date,Name,Shares,PricePerShare,SharesOwnedFollowing,fileDate,total_value,log_total_value,TraderFrequency,change_in_holdings,individual_transactions_per_trade,investors_per_trade,Purchase?
0,NTRA,2023-12-29,brophy michael burkes,1377.0,62.2900,64810.0,2023-12-30 02:45:08,85773.3300,11.359463,2.0,2.124672,2.0,7.0,0.0
1,NTRA,2023-12-28,brophy michael burkes,915.0,63.2923,66187.0,2023-12-30 02:45:08,57912.4545,10.966688,2.0,1.382447,2.0,7.0,0.0
2,EAF,2023-12-29,bcp gp ltd,34111.0,2.2164,27370523.0,2023-12-30 02:38:57,75603.6204,11.233259,150.0,0.124627,150.0,1.0,0.0
3,EAF,2023-12-29,bcp gp ltd,2.0,2.2164,27275612.0,2023-12-30 02:38:57,4.4328,1.489031,150.0,0.000007,150.0,1.0,0.0
4,EAF,2023-12-29,bcp gp ltd,3.0,2.2164,27275614.0,2023-12-30 02:38:57,6.6492,1.894497,150.0,0.000011,150.0,1.0,0.0


In [ ]:
insiders.to_csv('insiders_cleaned_unprocessed.csv', index=False)

In [ ]:
import datetime as dt
def get_future_prices(row, days=[7, 14, 21, 28, 35, 42, 49, 56, 63, 70]):
    try:
        prices = pd.read_csv(f'../data/av_query_{row["Ticker"]}.csv', skiprows=5)
        prices['Date'] = pd.to_datetime(prices['5. Time Zone'])
        prices.drop(columns=['US/Eastern', '5. Time Zone'], inplace=True)
        prices['Close'] = prices['Unnamed: 2'].apply(lambda x: float(json.loads(x.replace("'", '"'))['4. close'].replace(',', '')))
        pricepoints = []
        for day in days:
            if (row['Date'] + dt.timedelta(day)) in prices['Date'].values:
                pricepoints.append(prices[prices['Date'] == (row['Date'] + dt.timedelta(day)) ]['Close'].values[0])
            else:
                pricepoints.append(np.nan)
        return pricepoints
    except:
        return [np.nan for day in days]

In [ ]:
import datetime

insiders[insiders['Date'] < (datetime.datetime(2024, 1, 1) - datetime.timedelta(days=100))]['Date'].count()
insiders_100d = insiders[insiders['Date'] > (datetime.datetime(2023, 12, 31) - datetime.timedelta(days=100))]
insiders_100d.info()
insiders_100d_over_med = insiders_100d[insiders_100d['total_value'] > insiders_100d['total_value'].median()]
insiders_100d_over_med.info()

In [12]:
entry = prices.iloc[0]['Unnamed: 2']
print(entry)


{'1. open': '29.9500', '2. high': '30.0700', '3. low': '29.4800', '4. close': '29.6800', '5. volume': '1026479'}


In [22]:

price_dict = json.loads(str(entry).replace("'", '"'), parse_float=lambda x: float(x.replace("'", '')))
print(price_dict)


{'1. open': '29.9500', '2. high': '30.0700', '3. low': '29.4800', '4. close': '29.6800', '5. volume': '1026479'}


In [23]:
isinstance(price_dict['4. close'], str)

True

In [25]:
close = float(price_dict['4. close'].replace("'", ''))
print(close)
print(type(close))

29.68
<class 'float'>


In [9]:
#prices['Close'] = prices['PriceMarks'].apply(lambda x: float(x.split('.')[4].split("'")[2]))
prices.head()

,5. Time Zone,Unnamed: 2
0,2024-01-26,"{'1. open': '29.9500', '2. high': '30.0700', '..."
1,2024-01-25,"{'1. open': '29.0200', '2. high': '29.8200', '..."
2,2024-01-24,"{'1. open': '29.4300', '2. high': '29.4300', '..."
3,2024-01-23,"{'1. open': '29.5500', '2. high': '29.6400', '..."
4,2024-01-22,"{'1. open': '29.1500', '2. high': '29.5700', '..."
